# Load libs

In [1]:
import random
from datetime import datetime

import os
import numpy as np
import pandas as pd 
from skimage.io import imread
from skimage.morphology import label
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from skimage.morphology import binary_opening, disk

from keras.utils import Sequence
from keras.models import load_model

Using TensorFlow backend.


In [6]:
import keras
print("keras:", keras.__version__)

import skimage 
print("skimage:", skimage.__version__)

import matplotlib
print("matplotlib:", matplotlib.__version__)

import tqdm
print("tqdm:", tqdm.__version__)

import numpy
print("numpy:", numpy.__version__)

import pandas
print("pandas:", pandas.__version__)


keras: 2.2.4
skimage: 0.13.0
matplotlib: 2.2.3
tqdm: 4.26.0
numpy: 1.15.2
pandas: 0.23.4


AttributeError: module 'datetime' has no attribute '__version__'

In [ ]:
PRETRAINED_DETECTOR_PATH = '../input/airbus-ship-detection-resnet34-256px/ResNet34_ship_detection_256_4.h5'
PRETRAINED_SEG_MODEL_PATH = '../input/airbus-seg-model-768-sr-0741/seg_model_768_ships_ratio_0741_5hr.h5'

SHIP_DET_DF = '../input/airbus-has-ship-or-not/has_ship_or_not.csv'

SHIP_DIR = '../input/airbus-ship-detection'
TRAIN_IMAGE_DIR = os.path.join(SHIP_DIR, 'train_v2')
TEST_IMAGE_DIR = os.path.join(SHIP_DIR, 'test_v2')

MASKS_DF_PATH = 'train_ship_segmentations_v2.csv'  # for vis. train GT

# Predict ship detection on images

In [ ]:
def multi_rle_encode(img):
    labels = label(img[:, :, 0])
    return [rle_encode(labels==k) for k in np.unique(labels[labels>0])]

def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def rle_decode(mask_rle, shape=(768, 768)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background
    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T  # Needed to align to RLE direction

def masks_as_image(in_mask_list):
    # Take the individual ship masks and create a single mask array for all ships
    all_masks = np.zeros((768, 768), dtype = np.int16)
    #if isinstance(in_mask_list, list):
    for mask in in_mask_list:
        if isinstance(mask, str):
            all_masks += rle_decode(mask)
    return np.expand_dims(all_masks, -1)


class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, test_names, batch_size=32, resize_to=None):
        'Initialization'
        self.test_names = test_names
        self.batch_size = batch_size
        self.resize_to = resize_to

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.test_names) / self.batch_size))
 
    def __data_generation(self, test_names_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, self.resize_to, self.resize_to, 3), dtype='float32')
#         y = np.empty((self.batch_size, self.resize_to, self.resize_to, 1), dtype='uint8')
        
        # Generate data
        for i, img_name in enumerate(test_names_temp):
            rgb_path = os.path.join(TEST_IMAGE_DIR, img_name)
            img = Image.open(rgb_path).convert('RGB')
            if self.resize_to is not None:
                img = img.resize((self.resize_to,self.resize_to))
            img = np.array(img)/255.0
                
            # Store sample
            X[i,] = img
            # Store class
#             y[i] = c_mask
        return X#, y
    
    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        test_names_temp = self.test_names[index*self.batch_size:(index+1)*self.batch_size]
        # Generate data
        X = self.__data_generation(test_names_temp)  # ~1.5s
        return X

# Main function

In [ ]:
class ShipSegmentator(object):
    """
    Class is responsible for test data ship segmentation having trained segmentation model
        and detection dataframe (has_ship=1/2 for every test image) or trained detector
    """
    def __init__(self, pretrained_seg_path, ship_dir=SHIP_DIR, pretrained_det_path=None, ship_det_df_path=SHIP_DET_DF, masks_df_path=MASKS_DF_PATH):
        """
        @pretrained_seg_path (str): path to pretrained segmentation model
        @ship_images_dir (str): path to folder that has subfolder 'test_v2' with test images
        @pretrained_det_path (str): path to pretrained deteciton model path (model predicts the probability of a ship in the image)
        @ship_det_df_path (str): SHIP_DET_DF): path to csv with df: columns=[ImageId, has_ship(0/1)]
        """
        self.train_image_dir = os.path.join(ship_dir, 'train_v2')
        self.test_image_dir = os.path.join(ship_dir, 'test_v2')
        self.test_names = os.listdir(self.test_image_dir)
        self.ship_det_df_path = ship_det_df_path
        self.pretrained_det_path = pretrained_det_path
        self.masks = pd.read_csv(os.path.join(ship_dir, masks_df_path))
        
        self.seg_model = load_model(pretrained_seg_path, compile=False)
        print("Segmentation model loaded!")

        self._split_ship_no_ship_images()
    
    def _split_ship_no_ship_images(self):
        if self.ship_det_df_path is None:
            print("File has_ship_or_not csv doesn't exist. \nPredict them using detector ->")
            det_df = self.predict_ship_exist_in_image(self.pretrained_det_path)
        else:
            print("Opening has_ship_or_not csv file...")
            det_df = pd.read_csv(self.ship_det_df_path)
            
        self.test_names_ship = det_df.loc[det_df['has_ship'] >= 0.5, ['ImageId']]['ImageId'].values.tolist()
        self.test_names_nothing = det_df.loc[det_df['has_ship'] < 0.5, ['ImageId']]['ImageId'].values.tolist()
        print("'Ship' images:{}, 'no ship' images:{}".format(len(self.test_names_ship), len(self.test_names_nothing)))
            
        
    def predict_ship_exist_in_image(self, pretrained_det_path):
        det_model = load_model(pretrained_det_path, compile=False)
        test_generator = DataGenerator(self.test_names, batch_size=54, resize_to=256)  # 15606/54 = int, so last batch included
        print("Ship on images prediction takes about 5 mins...")
        result = det_model.predict_generator(test_generator)[:, 0]
        result = (result >= 0.5).astype(int)
        ship_det_df = pd.DataFrame({
            "ImageId": test_names,
            "has_ship":result
        })
        ship_det_df.to_csv('has_ship_or_not.csv',index = False)
        return ship_det_df

    def visualize_ship_segmentation(self, n_images=10, is_train=False, shuffle=False):
        if is_train:
            # pick only ship exist images
            img_names_list = self.masks[~self.masks['EncodedPixels'].isna()]['ImageId'].unique()
            n_columns = 3
            images_dir = self.train_image_dir
        else:
            img_names_list = self.test_names_ship
            n_columns = 2
            images_dir = self.test_image_dir
        if shuffle:
            random.shuffle(img_names_list)
        
        fig, m_axs = plt.subplots(n_images, n_columns, figsize = (5*n_columns, 4*n_images))
        for axes, img_name in zip(m_axs, img_names_list):
            path = os.path.join(images_dir, img_name)
            img = imread(path)
            img = np.expand_dims(img, 0)/255.0
            seg_mask = self.seg_model.predict(img)

            
            axes[0].imshow(img[0])
            axes[0].set_title(img_name)
            axes[1].imshow(seg_mask[0, :, :, 0], vmin = 0, vmax = 1)
            axes[1].set_title('Prediction')
            if is_train:
                true_mask = masks_as_image(self.masks[self.masks['ImageId']==img_name]['EncodedPixels'].values)
                axes[2].imshow(true_mask[:,:,0], cmap='gray')
                axes[2].set_title('Ground Truth')
        plt.show()
#         fig.savefig('test_predictions.png')

    def predict_ship_masks_to_csv(self):
        """
        Main function that predict mask for images
        classified as 'has ship' by the classifier. 
        Encode RLEs to submission file if mask predicted,
        if model predicts empty mask that also put NaN to 'EncodedPixels' columns
        as for 'no ship' images from classifier(detector)
        """
        # put 'no ship' images to final sub dict
        ship_list_dict = []
        for name in test_names_nothing:
            ship_list_dict.append({'ImageId':name,'EncodedPixels':None})
              
        print("Start segmentation prediction:", datetime.now())
        for img_name in tqdm_notebook(test_names_ship):
            if img_name in test_names:
                path = os.path.join(TEST_IMAGE_DIR, img_name)
                img = imread(path)
                img = np.expand_dims(img, 0)/255.0
                cur_seg = self.seg_model.predict(img)[0]
                cur_seg = binary_opening(cur_seg>=0.5, np.expand_dims(disk(2), -1))
                cur_rles = multi_rle_encode(cur_seg)
                if len(cur_rles)>0:
                    for c_rle in cur_rles:
                        ship_list_dict += [{'ImageId': img_name, 'EncodedPixels': c_rle}]
                else:
                    ship_list_dict += [{'ImageId': img_name, 'EncodedPixels': None}]
            else:
                print("img_name is not in self.test_names. img_name:", img_name)
            gc.collect()
              
        submission_df = pd.DataFrame(ship_list_dict)[['ImageId', 'EncodedPixels']]
        submission_df.to_csv('submission.csv', index=False)
        return submission_df

In [ ]:
ship_segmentator = ShipSegmentator(pretrained_seg_path=PRETRAINED_SEG_MODEL_PATH, ship_dir=SHIP_DIR, pretrained_det_path=None, ship_det_df_path=SHIP_DET_DF)

In [ ]:
ship_segmentator.visualize_ship_segmentation(n_images=16, is_train=False, shuffle=True)